In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from birdCLEFloaddata import load_audiofile,load_metadata,get_melspectrogram
from birdCLEFFunctions import Dynamic_CNN, Dynamic_CNN2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import Resize
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import soundfile as sf

plt.rcParams['figure.dpi'] = 600


In [2]:
torch.cuda.is_available()

False

In [3]:
#load data
path = r"C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCLEFproject\data\birdCLEF2023"

meta_data = load_metadata(path)
meta_data.head(10)


,latitude,longitude,common_name,rating,filename
0,4.3906,38.2788,African Bare-eyed Thrush,4.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
1,-2.9524,38.2921,African Bare-eyed Thrush,3.5,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
2,-2.9524,38.2921,African Bare-eyed Thrush,3.5,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
3,-2.9524,38.2921,African Bare-eyed Thrush,5.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
4,-2.9524,38.2921,African Bare-eyed Thrush,4.5,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
5,-2.9524,38.2921,African Bare-eyed Thrush,3.5,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
6,-2.9965,37.6244,African Bare-eyed Thrush,3.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
7,-4.0904,37.8807,African Bare-eyed Thrush,5.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
8,-4.0904,37.8807,African Bare-eyed Thrush,5.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...
9,4.8403,38.6988,African Bare-eyed Thrush,4.0,C:\Users\zhakk\Desktop\Uni\Kandidat\AML-BirdCL...


In [4]:
print(len(set(meta_data['common_name'])))
print()


264



In [5]:
print(len(set(meta_data['common_name'].unique())))

264


In [6]:
pd.set_option('display.max_rows', None)
print(meta_data['common_name'].value_counts())

Barn Swallow                        500
Willow Warbler                      500
Thrush Nightingale                  500
Western Yellow Wagtail              500
Common Sandpiper                    500
Wood Sandpiper                      486
Common Buzzard                      477
European Bee-eater                  437
Eurasian Hoopoe                     436
Common House-Martin                 425
Little Egret                        378
Common Bulbul                       293
Red-backed Scrub-Robin              281
Black Kite                          262
Great Egret                         252
Gray-backed Camaroptera             239
Red-rumped Swallow                  227
Sombre Greenbul                     199
Collared Sunbird                    181
Rattling Cisticola                  172
Black-backed Puffback               166
Cattle Egret                        166
Tawny-flanked Prinia                161
Cape Robin-Chat                     153
Egyptian Goose                      152


In [7]:
def load_audiofile2(filepath, cutoff_time):
    #Function that takes a file and cutoff time to create training and validation sets for training
    #If the audioclip is lower than the cutoff_time then the clip is looped untill desired duration reached
    audio, sr = sf.read(filepath)
    duration = len(audio) / sr


    if duration >= cutoff_time:
        training_audio = audio[:int(sr * 15)]
        validation_audio = audio[int(sr * 15):int(sr * 30)]
    else:
        
        loop_count = int(np.ceil(cutoff_time / duration))
        audio = np.tile(audio, loop_count)

        training_audio = audio[:int(sr * 15)]
        validation_audio = audio[int(sr * 15):int(sr * 30)]

    return training_audio.astype(np.float32), validation_audio.astype(np.float32), sr

In [8]:
#Generate audio data
training_data = []
validation_data = []
for i in range(50):
    train_audio,validation_audio, sr = load_audiofile2(meta_data['filename'][i],30)
    training_data.append([get_melspectrogram(train_audio),meta_data['common_name'][i]])
    validation_data.append([get_melspectrogram(validation_audio),meta_data['common_name'][i]])


training_data = np.asarray(training_data)
validation_data = np.asarray(validation_data)
label_mapping = {label: index for index, label in enumerate(set(meta_data['common_name'].unique()))}
training_data[:,1] = [label_mapping.get(label, -1)+1 for label in training_data[:,1]]
validation_data[:,1] = [label_mapping.get(label, -1)+1 for label in validation_data[:,1]]


C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\1452550675.py:10: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  training_data = np.asarray(training_data)
C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\1452550675.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.asarray(training_data)
C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\145255

In [9]:
training_data

array([[tensor([[ -15.9328,   -1.5002,   23.2655,  ...,    8.1975,    4.0844,
                   -1.0061],
                [ -18.6061,   -1.2447,   11.9637,  ...,    1.3702,  -18.1093,
                  -16.5076],
                [ -19.5640,   -3.8997,   -3.6010,  ...,   -6.5982,  -16.7892,
                  -26.4965],
                ...,
                [ -21.8953,  -20.3882,  -10.3955,  ...,   -3.5575,   -2.2703,
                   -4.0791],
                [ -22.6447,  -27.8357,  -29.6963,  ...,  -21.0560,  -22.9366,
                  -19.5833],
                [ -28.4901,  -46.2883,  -65.3653,  ..., -169.6379,  -69.9517,
                  -32.4270]])                                                ,
        242],
       [tensor([[-21.7014, -24.2332, -36.9520,  ..., -71.5732, -53.8378, -49.4398],
                [-21.9937, -24.5209, -36.9520,  ..., -72.6069, -53.7806, -49.3549],
                [-22.4472, -24.9543, -36.9520,  ..., -70.6594, -53.7303, -49.2791],
                ...,


In [14]:
import random

#Generate audio data
training_data = []
validation_data = []
cutoff_time = 30

for common_name in meta_data['common_name'].unique():
    i = meta_data.loc[meta_data['common_name'] == common_name].index[0] 
    train_audio,validation_audio, sr = load_audiofile2(meta_data['filename'][i],cutoff_time)
    training_data.append([get_melspectrogram(train_audio,n_fft=2048,hop_length=512,n_mels=128,fmin=40,fmax=15000,power=2.0,top_db=100),meta_data['common_name'][i]])
    validation_data.append([get_melspectrogram(validation_audio,n_fft=2048,hop_length=512,n_mels=128,fmin=40,fmax=15000,power=2.0,top_db=100),meta_data['common_name'][i]])

#Random selection of 100 data points, each point is taken in groups of 5
selected_spots = random.sample(range(len(meta_data) - 4), 100)  

for i in selected_spots:
    for j in range(i, i + 4):
        train_audio,validation_audio, sr = load_audiofile2(meta_data['filename'][i],cutoff_time)
        training_data.append([get_melspectrogram(train_audio,n_fft=2048,hop_length=512,n_mels=128,fmin=40,fmax=15000,power=2.0,top_db=100),meta_data['common_name'][i]])
        validation_data.append([get_melspectrogram(validation_audio,n_fft=2048,hop_length=512,n_mels=128,fmin=40,fmax=15000,power=2.0,top_db=100),meta_data['common_name'][i]])


training_data = np.asarray(training_data)
validation_data = np.asarray(validation_data)
label_mapping = {label: index for index, label in enumerate(set(meta_data['common_name'].unique()))}
training_data[:,1] = [label_mapping.get(label, -1)+1 for label in training_data[:,1]]
validation_data[:,1] = [label_mapping.get(label, -1)+1 for label in validation_data[:,1]]

C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\1835328613.py:24: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  training_data = np.asarray(training_data)
C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\1835328613.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.asarray(training_data)
C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\183532

In [20]:
training_data = []
validation_data = []
cutoff_time = 30

for common_name in meta_data['common_name'].unique():
    i = meta_data.loc[meta_data['common_name'] == common_name].index[0] 
    train_audio,validation_audio, sr = load_audiofile2(meta_data['filename'][i],cutoff_time)
    training_data.append([get_melspectrogram(train_audio,n_fft=2048,hop_length=512,n_mels=128,fmin=40,fmax=15000,power=2.0,top_db=100),meta_data['common_name'][i]])
    validation_data.append([get_melspectrogram(validation_audio,n_fft=2048,hop_length=512,n_mels=128,fmin=40,fmax=15000,power=2.0,top_db=100),meta_data['common_name'][i]])

training_data = np.asarray(training_data)
validation_data = np.asarray(validation_data)
label_mapping = {label: index for index, label in enumerate(set(meta_data['common_name'].unique()))}
training_data[:,1] = [label_mapping.get(label, -1)+1 for label in training_data[:,1]]
validation_data[:,1] = [label_mapping.get(label, -1)+1 for label in validation_data[:,1]]

C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\105945342.py:11: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  training_data = np.asarray(training_data)
C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\105945342.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.asarray(training_data)
C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\10594534

## Spectrogram extraction

In [128]:
#Generate spectrograms
spectrograms = []
for i in range(50):
    spectrograms.append([get_melspectrogram(meta_data['filename'][i]),meta_data['common_name'][i]])
spectrograms = np.asarray(spectrograms)


C:\Users\zhakk\AppData\Local\Temp\ipykernel_24964\1658868461.py:5: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  spectrograms = np.asarray(spectrograms)
C:\Users\zhakk\AppData\Local\Temp\ipykernel_24964\1658868461.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spectrograms = np.asarray(spectrograms)


In [23]:
#load one of each birds data
spectrograms = []

for common_name in meta_data['common_name'].unique():
    index = meta_data.loc[meta_data['common_name'] == common_name].index[0] 
    spectrogram = get_melspectrogram(meta_data['filename'][index])  
    spectrograms.append([spectrogram, common_name])

spectrograms = np.asarray(spectrograms)

C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\1861822250.py:9: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  spectrograms = np.asarray(spectrograms)
C:\Users\zhakk\AppData\Local\Temp\ipykernel_24588\1861822250.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spectrograms = np.asarray(spectrograms)


In [ ]:
import random
#Random selection of 100 data points, each point is taken in groups of 5
spectrograms = []
selected_spots = random.sample(range(len(meta_data) - 4), 100)  

for i in selected_spots:
    for j in range(i, i + 4):
        spectrograms.append([get_melspectrogram(meta_data['filename'][j]), meta_data['common_name'][j]])

spectrograms = np.asarray(spectrograms)


In [28]:
#Generate audio data
training_data = []
validation_data = []
for i in range(50):
    train_audio,validation_audio, sr = load_audiofile(meta_data['filename'][i],35)
    training_data.append([get_melspectrogram(train_audio),meta_data['common_name'][i]])
    validation_data.append([get_melspectrogram(validation_audio),meta_data['common_name'][i]])

training_data = np.asarray(training_data)
validation_data = np.asarray(validation_data)

TypeError: load_audiofile() takes 1 positional argument but 2 were given

In [24]:
#Split data into validation and test. (Need to look at this again, possible mistake in the validation data creation)
train_ratio = 0.8
train_size = int(train_ratio * len(spectrograms))

train_labels = spectrograms[:,1]

label_mapping = {label: index for index, label in enumerate(set(train_labels))}

spectrograms[:,1] = [label_mapping.get(label, -1)+1 for label in train_labels]

validation_set = []
training_set = []

#Split data 
for i, (spectrogram, label) in enumerate(spectrograms):
    shape = np.shape(spectrogram)
    if shape[1] >= 100:
        validation_data = spectrogram[:, :50]
        validation_set.append([validation_data, label])
        
        remaining_data = spectrogram[:, 50:]
        num_chunks = remaining_data.shape[1] // 50
        if num_chunks > 0:
            chunks = np.split(remaining_data[:, :num_chunks*50], num_chunks, axis=1)
            for chunk in chunks:
                training_set.append([chunk, label])
    else: print(i,label)


## Network

### From audiofiles

In [21]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(0, num_classes)  # Initialize with size 0
        self.fc2 = nn.Linear(num_classes, num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        size = x.shape[1] * x.shape[2] * x.shape[3]
        x = x.view(x.size(0), -1)
        self.fc1 = nn.Linear(size, num_classes)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [22]:

#Load data into batches of 32
batch_size = 10
train_loader = DataLoader(training_data.tolist(), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_data.tolist(), batch_size=batch_size, shuffle=False)

# Initialize the CNN and use +1 for classes due to "no class" being labeled as -1
num_classes = len(meta_data['common_name'].unique())+1
cnn = CNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

# Training loop and attempt to use cuda
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn.to(device)

for epoch in range(num_epochs):
    print("epoch : ", epoch)
    cnn.train()
    train_loss = 0.0
    train_correct = 0
    i = 0 
    for images, labels in train_loader:
        #load data onto device, either gpu or cpu
        images = images.unsqueeze(1).to(device) 
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        i+=1
        
    # Validation loop
    cnn.eval()
    val_loss = 0.0
    val_correct = 0
    
    with torch.no_grad(): #Disables some calculations, used to reduce memory.
        for images, labels in val_loader:
        #load data onto device, either gpu or cpu
            images = images.unsqueeze(1).to(device)
            labels = labels.to(device)
            
            outputs = cnn(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
    
    train_loss /= len(train_loader.dataset)
    train_acc = train_correct / len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)
    val_acc = val_correct / len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

epoch :  0
Epoch 1/10:
Train Loss: 5.7797 | Train Acc: 0.0000
Val Loss: 5.5865 | Val Acc: 0.0038
epoch :  1
Epoch 2/10:
Train Loss: 5.5861 | Train Acc: 0.0000
Val Loss: 5.5724 | Val Acc: 0.0076
epoch :  2
Epoch 3/10:
Train Loss: 5.5898 | Train Acc: 0.0000
Val Loss: 5.5753 | Val Acc: 0.0000
epoch :  3
Epoch 4/10:
Train Loss: 5.5784 | Train Acc: 0.0076
Val Loss: 5.5632 | Val Acc: 0.0076
epoch :  4
Epoch 5/10:
Train Loss: 5.5650 | Train Acc: 0.0000
Val Loss: 5.5414 | Val Acc: 0.0038
epoch :  5
Epoch 6/10:
Train Loss: 5.5595 | Train Acc: 0.0114
Val Loss: 5.5367 | Val Acc: 0.0038
epoch :  6
Epoch 7/10:
Train Loss: 5.5210 | Train Acc: 0.0000
Val Loss: 5.4806 | Val Acc: 0.0038
epoch :  7
Epoch 8/10:
Train Loss: 5.5187 | Train Acc: 0.0038
Val Loss: 5.4725 | Val Acc: 0.0038
epoch :  8
Epoch 9/10:
Train Loss: 5.4958 | Train Acc: 0.0000
Val Loss: 5.4477 | Val Acc: 0.0076
epoch :  9


KeyboardInterrupt: 

### Direct Spectrogram

In [25]:
#Working
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(12288, num_classes)
        self.fc2 = nn.Linear(num_classes, num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [28]:

#Load data into batches of 32
batch_size = 32
train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=False)

# Initialize the CNN and use +1 for classes due to "no class" being labeled as -1
num_classes = len(meta_data['common_name'].unique())+1
cnn = CNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

# Training loop and attempt to use cuda
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn.to(device)

for epoch in range(num_epochs):
    print("epoch : ", epoch)
    cnn.train()
    train_loss = 0.0
    train_correct = 0
    i = 0 
    for images, labels in train_loader:
        #load data onto device, either gpu or cpu
        images = images.unsqueeze(1).to(device) 
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        i+=1
        
    # Validation loop
    cnn.eval()
    val_loss = 0.0
    val_correct = 0
    
    with torch.no_grad(): #Disables some calculations, used to reduce memory.
        for images, labels in val_loader:
        #load data onto device, either gpu or cpu
            images = images.unsqueeze(1).to(device)
            labels = labels.to(device)
            
            outputs = cnn(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
    
    train_loss /= len(train_loader.dataset)
    train_acc = train_correct / len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)
    val_acc = val_correct / len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

epoch :  0
Epoch 1/10:
Train Loss: 2.4425 | Train Acc: 0.5558
Val Loss: 1.8800 | Val Acc: 0.6136
epoch :  1
Epoch 2/10:
Train Loss: 0.2779 | Train Acc: 0.9243
Val Loss: 1.3604 | Val Acc: 0.6894
epoch :  2
Epoch 3/10:
Train Loss: 0.1257 | Train Acc: 0.9667
Val Loss: 1.4042 | Val Acc: 0.7311
epoch :  3
Epoch 4/10:
Train Loss: 0.0935 | Train Acc: 0.9751
Val Loss: 1.0784 | Val Acc: 0.7689
epoch :  4
Epoch 5/10:
Train Loss: 0.0676 | Train Acc: 0.9790
Val Loss: 1.0558 | Val Acc: 0.7992
epoch :  5
Epoch 6/10:
Train Loss: 0.0330 | Train Acc: 0.9908
Val Loss: 1.0194 | Val Acc: 0.8106
epoch :  6
Epoch 7/10:
Train Loss: 0.0328 | Train Acc: 0.9903
Val Loss: 1.4972 | Val Acc: 0.7652
epoch :  7
Epoch 8/10:
Train Loss: 0.1248 | Train Acc: 0.9660
Val Loss: 1.8597 | Val Acc: 0.7045
epoch :  8
Epoch 9/10:
Train Loss: 0.0658 | Train Acc: 0.9823
Val Loss: 1.5548 | Val Acc: 0.7311
epoch :  9
Epoch 10/10:
Train Loss: 0.0647 | Train Acc: 0.9860
Val Loss: 1.6895 | Val Acc: 0.7197


In [29]:
num_classes

265

## Plots